In [3]:
# Importacao das bibliotecas

import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [4]:
# Lendo o arquivo

df = pd.read_csv('/content/dataframe.csv')
df.sample(5)

,data_medicao,nome_condominio,quantidade_placas,potencia_placa_kW,consumo_kWh,quantidade_habitantes,energia_solar_gerada_kWh,energia_recomendada_kWh
54106,2026-12-18,Condominio_7,25,0.30,1792.73,143,37.11,5975.78
86997,2028-10-05,Condominio_48,51,0.30,5772.48,483,66.00,19241.60
54117,2026-12-18,Condominio_18,36,0.35,5419.09,415,53.54,18063.63
61491,2027-05-14,Condominio_42,47,0.35,5091.84,404,57.45,16972.82
144199,2031-11-23,Condominio_50,44,0.35,5121.73,464,62.12,17072.42


In [5]:
# Vendo informações sobre o dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   data_medicao              150000 non-null  object 
 1   nome_condominio           150000 non-null  object 
 2   quantidade_placas         150000 non-null  int64  
 3   potencia_placa_kW         150000 non-null  float64
 4   consumo_kWh               150000 non-null  float64
 5   quantidade_habitantes     150000 non-null  int64  
 6   energia_solar_gerada_kWh  150000 non-null  float64
 7   energia_recomendada_kWh   150000 non-null  float64
dtypes: float64(4), int64(2), object(2)
memory usage: 9.2+ MB


## Detalhes do dataset

Podemos verificar que o dataset é bem completo e não possui colunas nulas, isso vai ajudar a treinar nosso modelo.

### Informações sobre as colunas

 - data_medicao  
   A data que foi tirado aquela medição.

 - nome_condominio  
   O nome do condomínio.

 - quantidade_placas  
   A quantidade de placas que o condomínio tinha no dia que foi tirado aquela medição.

 - potencia_placa_kW  
   A potência daquela placa no dia daquela medição.

 - consumo_kWh:  
   O consumo de energia do condomínio naquele dia.

 - quantidade_habitantes:  
   A quantidade de habitantes que tinha no condomínio naquele dia.

 - energia_solar_gerada_kWh:  
   A energia que estava sendo gerada naquele dia.

 - energia_recomendada_kWh:  
   A quantidade de energia recomendada



In [6]:
df.head()

,data_medicao,nome_condominio,quantidade_placas,potencia_placa_kW,consumo_kWh,quantidade_habitantes,energia_solar_gerada_kWh,energia_recomendada_kWh
0,2024-01-01,Condominio_1,43,0.35,5517.13,383,67.54,18390.43
1,2024-01-01,Condominio_2,41,0.30,2564.63,221,54.79,8548.75
2,2024-01-01,Condominio_3,24,0.30,5653.48,451,32.49,18844.92
3,2024-01-01,Condominio_4,49,0.35,1426.67,142,78.65,4755.58
4,2024-01-01,Condominio_5,47,0.30,2501.70,211,61.23,8339.00


In [20]:
# Criando coluna para marcar a estação do ano

# Primeiro vamos transformar a coluna em uma data que o python consiga entender
df['data_medicao'] = pd.to_datetime(df['data_medicao'])

# Extraindo o mês para classificar a estação do ano
df['mes'] = df['data_medicao'].apply(lambda x: x.month)
df['mes'].value_counts()

# Aplicando a função para marcar a estação do ano
def get_estacao(mes):
    if mes in [12, 1, 2]:
        return 'Verão'
    elif mes in [3, 4, 5]:
        return 'Outono'
    elif mes in [6, 7, 8]:
        return 'Inverno'
    else:
        return 'Primavera'

df['estacao'] = df['mes'].apply(get_estacao)
df['estacao'].value_counts()


,count
estacao,
Verão,39100
Outono,37700
Inverno,36800
Primavera,36400
